# Trabajo Integrador Comunicaciones Digitales


## Diseño del Codificador-Decodificador

## Implementación de Codificador y Decodificador LoRa según Vangelista

Utilizando una Jupyter Notebook, implementar el codificador de la ecuación 1 propuesto en el paper de Vangelista y su correspondiente decodificador. El diseño debe permitir enviar una cantidad de bits múltiplo del Spreading Factor (SF). Estos bits tienen que ser aleatorios con una función de distribución de probabilidad uniforme. El script debe permitir imprimir una parte de los bits transmitidos y los bits decodificados. A la salida del decodificador se debe calcular e imprimir la probabilidad de error de bit (BER), entendiendose como la relación entre la cantidad de bits errados sobre la cantidad de bits enviados. Utilizar una celda de la Jupyter Notebook para desarrollar la matematica y/o lógica utilizada en el algoritmo propuesto por ud.

### Codificador (Ecuación 1 del paper)

La Ecuación (1) del paper define la conversión de un vector de `SF` bits en un entero $s \in \{0, 1, \dots, 2^{SF} - 1\}$ mediante:

$$
s(nT_s) = \sum_{h=0}^{SF-1} w(nT_s)_h \cdot 2^h
$$

donde $w(nT_s)_h$ representa el bit de menor peso en $h = 0$ y el de mayor peso en $h = SF - 1$.

Este valor entero $s$ representa el símbolo que será modulado usando un chirp en LoRa. En esta celda lo tratamos solo como conversión lógica de bits a enteros.

---

### Decodificador

Dado un símbolo $s$, podemos recuperar el vector de bits original aplicando:

$$
w(nT_s)_h = \text{bit en la posición } h \text{ del número } s
$$

Esto puede implementarse utilizando operaciones de *bit-shift* y *AND*, o simplemente con funciones como `format(s, '0{SF}b')`.

---

### Bit Error Rate (BER)

La tasa de error de bit (Bit Error Rate) se calcula como:

$$
\text{BER} = \frac{\text{Número de bits erróneos}}{\text{Total de bits transmitidos}}
$$




In [ ]:
import numpy as np

# Parámetros
SF = 7
num_simbolos = 10       # Usamos 10 símbolos para ver bien los datos
total_bits = num_simbolos * SF

# Bits aleatorios
bits_tx = np.random.randint(0, 2, total_bits, dtype=np.uint8)

# Codificación vectorizada: bits → símbolos
bits_reshaped = bits_tx.reshape(-1, SF)                      # (N, SF)
potencias = 2**np.arange(SF, dtype=np.uint16)                # [1, 2, 4, ..., 2^(SF-1)]
simbolos_tx = np.dot(bits_reshaped, potencias)               # (N,)

# Decodificación vectorizada: símbolos → bits
simbolos_expand = simbolos_tx[:, None]                       # (N, 1)
bits_rx = ((simbolos_expand >> np.arange(SF)) & 1).astype(np.uint8)  # (N, SF)
bits_rx_flat = bits_rx.reshape(-1)                           # (N*SF,)

# Mostrar comparación de entrada y salida
print("Bits originales por símbolo (TX):")
print(bits_reshaped)

print("\nSímbolos codificados:")
print(simbolos_tx)

print("\nBits decodificados por símbolo (RX):")
print(bits_rx)

# Verificación de BER
BER = np.mean(bits_tx != bits_rx_flat)
print(f"\nBER: {BER:.10f}")
assert BER == 0.0, "Error: BER no es cero. Revisa la implementación."

Bits originales por símbolo (TX):
[[0 1 1 0 0 0 1]
 [1 1 1 1 0 0 1]
 [0 1 1 0 1 1 1]
 [1 0 0 0 0 0 0]
 [1 0 1 1 0 1 1]
 [1 0 0 1 1 0 0]
 [0 0 1 1 1 1 0]
 [1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1]
 [0 0 0 0 0 1 1]]

Símbolos codificados:
[ 70  79 118   1 109  25  60  63 127  96]

Bits decodificados por símbolo (RX):
[[0 1 1 0 0 0 1]
 [1 1 1 1 0 0 1]
 [0 1 1 0 1 1 1]
 [1 0 0 0 0 0 0]
 [1 0 1 1 0 1 1]
 [1 0 0 1 1 0 0]
 [0 0 1 1 1 1 0]
 [1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1]
 [0 0 0 0 0 1 1]]

BER: 0.0000000000


## Waveform Former y n-Tuple Former según Vangelista

Agregar a la Jupyter Notebook utilizada anteriormente, la implementación del waveform Former propuesto en la ecuación 2 del paper de Vangelista y su correspondiente n-Tuple Former descripto en la sección III. Con el agregado de esta etapa el diseño debe permitir enviar una cantidad de bits múltiplo del Spreading Factor (SF). El script debe permitir imprimir una parte de los símbolos transmitidos y los símbolos decodificados. A la salida del n-Tuple former se debe calcular e imprimir la probabilidad de error de símbolo (SER), entendiéndose como la relación entre la cantidad de símbolos errados sobre la cantidad de símbolos enviados. Utilizar una celda de la Jupyter Notebook para desarrollar la matematica y/o lógica utilizada en el algoritmo propuesto por ud.



### Waveform Former (Ecuación 2 del paper)

El "waveform former" genera una señal chirp discreta asociada a un símbolo $s(nT_s)$ de duración $T_s = 2^{SF} \cdot T$. La señal transmitida es:

$$
c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot \exp\left( j 2\pi \cdot \frac{(s(nT_s) + k) \mod 2^{SF}}{2^{SF}} \cdot k \right)
$$

donde $k = 0, 1, ..., 2^{SF} - 1$ y $T = \frac{1}{B}$.

Esto genera una base ortonormal de señales chirp para cada símbolo $s \in \{0, ..., 2^{SF} - 1\}$.

---

### n-Tuple Former (Demodulador Óptimo)

Para demodular los símbolos recibidos, se realiza:

1. Multiplicación punto a punto de la señal recibida $r(k)$ con un **downchirp** de referencia:
   $$
   d(k) = r(k) \cdot \exp\left(-j 2\pi \cdot \frac{k^2}{2^{SF}} \right)
   $$

2. Cálculo de la **Transformada Discreta de Fourier (FFT)** del resultado $d(k)$.

3. El índice de la componente de mayor magnitud indica el símbolo transmitido:
   $$
   \hat{s} = \arg\max_{m} |\text{FFT}(d)[m]|
   $$

Este método implementa una detección eficiente mediante correlación en el dominio de la frecuencia.

---

### Probabilidad de Error de Símbolo (SER)

Se define como:

$$
SER = \frac{\text{Número de símbolos erróneos}}{\text{Total de símbolos transmitidos}}
$$


In [5]:
import numpy as np

# Parámetros
SF = 7
M = 2**SF
num_simbolos = 100  # Múltiplo de SF
total_bits = num_simbolos * SF

# Bits aleatorios
bits_tx = np.random.randint(0, 2, total_bits, dtype=np.uint8)

# Codificador lógico: bits → símbolos
bits_reshaped = bits_tx.reshape(-1, SF)                      # (N, SF)
potencias = 2**np.arange(SF, dtype=np.uint16)                # [1, 2, ..., 2^(SF-1)]
simbolos_tx = np.dot(bits_reshaped, potencias)               # (N,)

# Construir matriz de fases (modulación chirp)
symbol_matrix = (simbolos_tx[:, None] + k[None, :]) % M      # (N, M)
phase_matrix = 2 * np.pi * symbol_matrix * k[None, :] / M    # (N, M)
chirps_tx = (1 / np.sqrt(M)) * np.exp(1j * phase_matrix)     # (N, M)


# Canal ideal (sin ruido)
chirps_rx = chirps_tx.copy()

# n-Tuple Former (demodulación por correlación FFT)
# Paso 1: multiplicar por downchirp
downchirp = np.exp(-1j * 2 * np.pi * k**2 / M)
chirps_multiplicados = chirps_rx * downchirp                # Element-wise, shape (N, M)

# Paso 2: aplicar FFT
fft_output = np.fft.fft(chirps_multiplicados, axis=1)

# Paso 3: detectar símbolo como índice del máximo
simbolos_rx = np.argmax(np.abs(fft_output), axis=1)

# Mostrar ejemplo
print("Símbolos transmitidos:   ", simbolos_tx[:10])
print("Símbolos decodificados:  ", simbolos_rx[:10])

# Cálculo de SER
SER = np.mean(simbolos_tx != simbolos_rx)
print(f"\nSymbol Error Rate (SER): {SER:.10f}")
assert SER == 0.0, "Error: SER no es cero. Verifique implementación."


Símbolos transmitidos:    [ 65  52  62  54  35  31  86 120  67 113]
Símbolos decodificados:   [ 65  52  62  54  35  31  86 120  67 113]

Symbol Error Rate (SER): 0.0000000000
